In [1]:
% cd /home/mayu-ot/Documents/iparaphrasing-master/

/home/mayu-ot/Documents/iparaphrasing-master


In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics

In [3]:
val_df = pd.read_csv('models/fv+cca-vgg/res_val.csv')
test_df = pd.read_csv('models/fv+cca-vgg/res_test.csv')

In [4]:
# split validation set
val_mult_idx = []
val_single_idx = []

for i, row in val_df.iterrows():
    phr1 = row['phrase1']
    phr2 = row['phrase2']
    
    if ('+' in phr1) and ('+' in phr2):
        val_mult_idx.append(i)
    elif ('+' not in phr1) and ('+' not in phr2):
        val_single_idx.append(i)

# split test set
test_mult_idx = []
test_single_idx = []

for i, row in test_df.iterrows():
    phr1 = row['phrase1']
    phr2 = row['phrase2']
    
    if ('+' in phr1) and ('+' in phr2):
        test_mult_idx.append(i)
    elif ('+' not in phr1) and ('+' not in phr2):
        test_single_idx.append(i)

In [5]:
# eval on single word examples
single_examples = val_df.iloc[val_single_idx]
score = single_examples.score
ytrue = single_examples.ytrue

prec, rec, thresh = sklearn.metrics.precision_recall_curve(y_true=ytrue, probas_pred=score)
f1 = 2 * (prec * rec) / (prec + rec)
f1[np.isnan(f1)] = 0.
best_i = f1.argmax()
best_thresh = thresh[best_i]

print('[Single word examples] prec: %.4f, rec: %.4f, f1: %.4f'%(prec[best_i], rec[best_i], f1[best_i]))

# eval_on multple word examples
single_examples = test_df.iloc[test_single_idx]
score = single_examples.score
ytrue = single_examples.ytrue

s_prec = sklearn.metrics.precision_score(y_true=ytrue, y_pred=score>best_thresh)
s_rec = sklearn.metrics.recall_score(y_true=ytrue, y_pred=score>best_thresh)
s_f1 = sklearn.metrics.f1_score(y_true=ytrue, y_pred=score>best_thresh)

print('[Single word examples] prec: %.4f, rec: %.4f, f1: %.4f'%(s_prec, s_rec, s_f1))

[Single word examples] prec: 0.8065, rec: 0.8210, f1: 0.8137
[Single word examples] prec: 0.8452, rec: 0.8185, f1: 0.8316


In [6]:
# eval on multiple word examples
mult_examples = val_df.iloc[val_mult_idx]
score = mult_examples.score
ytrue = mult_examples.ytrue

prec, rec, thresh = sklearn.metrics.precision_recall_curve(y_true=ytrue, probas_pred=score)
f1 = 2 * (prec * rec) / (prec + rec)
f1[np.isnan(f1)] = 0.
best_i = f1.argmax()
best_thresh = thresh[best_i]
print('[Multiple word examples] prec: %.4f, rec: %.4f, f1: %.4f'%(prec[best_i], rec[best_i], f1[best_i]))

# eval_on multple word examples
mult_examples = test_df.iloc[test_mult_idx]
score = mult_examples.score
ytrue = mult_examples.ytrue

m_prec = sklearn.metrics.precision_score(y_true=ytrue, y_pred=score>best_thresh)
m_rec = sklearn.metrics.recall_score(y_true=ytrue, y_pred=score>best_thresh)
m_f1 = sklearn.metrics.f1_score(y_true=ytrue, y_pred=score>best_thresh)

print('[Multiple word examples] prec: %.4f, rec: %.4f, f1: %.4f'%(m_prec, m_rec, m_f1))

[Multiple word examples] prec: 0.7964, rec: 0.8687, f1: 0.8310
[Multiple word examples] prec: 0.8028, rec: 0.8682, f1: 0.8343


In [7]:
# eval on all examples
score = val_df.score
ytrue = val_df.ytrue

prec, rec, thresh = sklearn.metrics.precision_recall_curve(y_true=ytrue, probas_pred=score)
f1 = 2 * (prec * rec) / (prec + rec)
f1[np.isnan(f1)] = 0.
best_i = f1.argmax()
best_thresh = thresh[best_i]

# eval_on multple word examples
score = test_df.score
ytrue = test_df.ytrue

a_prec = sklearn.metrics.precision_score(y_true=ytrue, y_pred=score>best_thresh)
a_rec = sklearn.metrics.recall_score(y_true=ytrue, y_pred=score>best_thresh)
a_f1 = sklearn.metrics.f1_score(y_true=ytrue, y_pred=score>best_thresh)

print('[All examples] prec: %.4f, rec: %.4f, f1: %.4f'%(a_prec, a_rec, a_f1))

[All examples] prec: 0.8251, rec: 0.8419, f1: 0.8334


In [8]:
print('prec: %.2f/%.2f/%.2f'% (100*a_prec, 100*s_prec, 100*m_prec))
print('rec: %.2f/%.2f/%.2f'% (100*a_rec, 100*s_rec, 100*m_rec))
print('f1: %.2f/%.2f/%.2f'% (100*a_f1, 100*s_f1, 100*m_f1))

prec: 82.51/84.52/80.28
rec: 84.19/81.85/86.82
f1: 83.34/83.16/83.43
